### Generate Spotify playlists using the Track2Vec and MP3ToVec (trained on Spotify) embeddings

In [1]:
import keras
from keras.models import load_model
import os
import numpy as np
import librosa
import pickle
from tqdm import tqdm
import requests
import pickle
import csv
from IPython.display import clear_output
from IPython.core.display import display, HTML

Using TensorFlow backend.


In [2]:
# key will be Spotify id
mp3tovecs = pickle.load(open('../Pickles_Spotify/spotifytovec.p', 'rb'))
tracktovecs = pickle.load(open('../Pickles_Spotify/tracktovec.p', 'rb'))
print('Read in tracks...')
tracks = {}
with open('popular_tracks.csv', "r", encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    for row in spamreader:
        columns = str(row)[2:-2].split(';')
        if columns[3] != '' and columns[3][:5] == 'https':
            tracks[columns[0]] = (columns[1] + ' - ' + columns[2], columns[3])
print(f'Number of tracks processed: {len(mp3tovecs)}')

Read in tracks...
Number of tracks processed: 78198


In [3]:
#pip install git+https://github.com/plamere/spotipy.git --upgrade

import spotipy
import spotipy.util as util

scope = 'playlist-modify-public'
username = 'teticio'
client_id='194086cb37be48ebb45b9ba4ce4c5936'
client_secret='fb9fb4957a9841fcb5b2dbc7804e1e85'
redirect_uri='https://www.attentioncoach.es/'

token = util.prompt_for_user_token(username,scope, client_id, client_secret, redirect_uri)
sp = spotipy.Spotify(token)

playlist_name = 'Test'
playlists = sp.user_playlists(username)
playlist_id = [playlist['id'] for playlist in playlists['items'] if playlist['name'] == playlist_name][0]

def spotify_playlist(username, playlist_id, track_details):
    if len(track_details[0]) == 2:
        track_ids = [track_detail[0] for track_detail in track_details]
    else:
        track_ids = [track_detail for track_detail in track_details]
    result = sp.user_playlist_replace_tracks(username, playlist_id, track_ids)
#    print(result)
    display(HTML(f'<iframe src="https://open.spotify.com/embed/user/{username}/playlist/{playlist_id}" width="100%" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>'))

In [4]:
def most_similar(mp3tovecs, weights, positive=[], negative=[], topn=5):
    if isinstance(positive, str):
        positive = [positive] # broadcast to list
    if isinstance(negative, str):
        negative = [negative] # broadcast to list
    similar = [('', 0)] * len(mp3tovecs[0])
    for i, mp3tovec in enumerate(mp3tovecs):
        mp3_vec_i = np.sum([mp3tovec[i] for i in positive] + [-mp3tovec[i] for i in negative], axis=0)
        for j, track_j in enumerate(mp3tovec):
            if track_j in positive or track_j in negative:
                continue
            mp3_vec_j = mp3tovec[track_j]
            cos_proximity = np.dot(mp3_vec_i, mp3_vec_j) / (np.linalg.norm(mp3_vec_i) * np.linalg.norm(mp3_vec_j))
            similar[j] = (track_j, similar[j][1] + weights[i] * cos_proximity)
    return sorted(similar, key=lambda x:-x[1])[:topn]

def make_playlist(mp3tovecs, weights, seed_tracks, size=10, lookback=3):
    playlist = seed_tracks
    while len(playlist) < size:
        candidates = most_similar(mp3tovecs, weights, positive=playlist[-lookback:], topn=5)
        for i in range(5):
            if not candidates[i][0] in playlist:
                break
        playlist.append(candidates[i][0])
    return playlist

def display_playlist(track_details):
    if len(track_details[0] == 2):
        for track_detail in track_details:
            display(HTML(f'<iframe src="https://open.spotify.com/embed/track/{track_detail[0]}" width="100%" height="80" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>'))
    else:
        for track_detail in track_details:
            display(HTML(f'<iframe src="https://open.spotify.com/embed/track/{track_detail}" width="100%" height="80" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>'))
    
def print_track_details(track_details):
    if len(track_details[0] == 2):
        for i, track_detail in enumerate(track_details):
            display(HTML(f'{i+1}. <a target="_blank" href="{tracks[track_detail[0]][1]}">{tracks[track_detail[0]][0]}</a> [{track_detail[1]:.2f}]'))
    else:
        for i, track_detail in enumerate(track_details):
            display(HTML(f'{i+1}. <a target="_blank" href="{tracks[track_detail[1]]}">{tracks[track_detail[0]][0]}</a> [{track_detail[1]:.2f}]'))

In [135]:
search = 'james brown'
ids = [track for track in mp3tovecs if all(word in tracks[track][0].lower() for word in search.lower().split())]
for id in ids:
    print(f'{id} : {tracks[id][0]}')

5rsnq2h3OLOg1FnW8m1trx : James Brown - It's a Man's, Man's, Man's World
0rTkE0FmT4zT2xL6GXwosU : James Brown & The Famous Flames - I Got You (I Feel Good)
58iP9J86ksOPwbo0pWOafk : James Brown - People Get Up And Drive Your Funky Soul - Remix
2q8eudK0r9ImgCB1XhFfxG : James Brown - Get Up Offa That Thing


### Add songs to an existing playlist and display it

In [6]:
id = '58iP9J86ksOPwbo0pWOafk'
spotify_playlist(username, playlist_id, make_playlist([mp3tovecs, tracktovecs], [0.5, 0.5], [id], size=20))

C:\Users\tetic\Anaconda3\envs\Deep\lib\site-packages\IPython\core\display.py:689: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


### You don't need a Spotify account for this to work, although you'll only be able to listen to 30s samples

In [59]:
id = '5SaZaoRLPVZDkcDd1szp04'
display(HTML('<h3>Most similar sounding</h3>'))
display_playlist([(id, 1)] + most_similar([mp3tovecs], [1], positive=[id], topn=5))
#display_playlist(make_playlist(mp3tovecs, [id]))
display(HTML('<h3>By similar artists</h3>'))
display_playlist([(id, 1)] + most_similar([tracktovecs], [1], positive=[id], topn=5))
#display_playlist(make_playlist(tracktovecs, [id]))
display(HTML('<h3>Recommended</h3>'))
display_playlist([(id, 1)] + most_similar([mp3tovecs, tracktovecs], [0.5, 0.5], positive=[id], topn=20))
#display_playlist(make_playlist(tracktovecs, [id]))

In [145]:
# save cut-down csv
spotify_tracks = {}
for track in tracks:
    spotify_tracks[track] = tracks[track][0]
pickle.dump(spotif_tracks, open('spotify_tracks.p', 'wb'))